In [8]:
import torch

# Đọc vocab
# with open("/kaggle/input/data-lstm/vocab3.txt", "r", encoding="utf-8") as f:
with open("/kaggle/input/data-small/vocab2.txt", "r", encoding="utf-8") as f:
    vocab = [line.strip() for line in f if line.strip()]

word2idx = {w: i for i, w in enumerate(vocab)}
idx2word = {i: w for i, w in enumerate(vocab)}

vocab_size = len(vocab)
print(f"Vocab size: {vocab_size}")


Vocab size: 23692


In [9]:
def encode_text(text, word2idx, unk_token="<unk>"):
    tokens = text.strip().split()
    return [word2idx.get(t, word2idx[unk_token]) for t in tokens]


In [10]:
# Đọc dữ liệu đã tokenized
# with open("/kaggle/input/data-lstm/train (3).txt", "r", encoding="utf-8") as f:
with open("/kaggle/input/data-small/train (2).txt", "r", encoding="utf-8") as f:
    train_text = f.read()

train_ids = encode_text(train_text, word2idx)
print(f"Số lượng token trong train: {len(train_ids):,}")


Số lượng token trong train: 2,367,821


In [11]:
from torch.utils.data import Dataset, DataLoader

class LSTMDataset(Dataset):
    def __init__(self, token_ids, seq_len=30):
        self.seq_len = seq_len
        self.data = []
        for i in range(0, len(token_ids) - seq_len):
            x = token_ids[i:i+seq_len]
            y = token_ids[i+1:i+seq_len+1]
            self.data.append((x, y))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x, y = self.data[idx]
        return torch.tensor(x), torch.tensor(y)

# Khởi tạo dataset
train_dataset = LSTMDataset(train_ids, seq_len=30)
# train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, drop_last=True)

print(f"Tổng số mẫu huấn luyện: {len(train_dataset):,}")


Tổng số mẫu huấn luyện: 2,367,791


In [11]:
import torch
import torch.nn.functional as F

def generate_text(model, seed_text, word2idx, idx2word, max_new_tokens=50, temperature=1.0):
    model.eval()
    input_ids = torch.tensor(
        [word2idx.get(w, word2idx["<unk>"]) for w in seed_text.split()],
        dtype=torch.long
    ).unsqueeze(0).to(device)

    hidden = None
    generated = seed_text.split()

    for _ in range(max_new_tokens):
        logits, hidden = model(input_ids, hidden)
        next_token_logits = logits[0, -1, :] / temperature
        probs = F.softmax(next_token_logits, dim=-1)

        # Lấy xác suất cao nhất
        max_prob = probs.max().item()

        # Tìm tất cả token có xác suất bằng xác suất cao nhất
        top_indices = (probs == max_prob).nonzero(as_tuple=True)[0]

        # Nếu có nhiều token cùng xác suất cao nhất → chọn ngẫu nhiên
        if len(top_indices) > 1:
            next_id = top_indices[torch.randint(len(top_indices), (1,))].item()
        else:
            next_id = top_indices.item()

        next_word = idx2word[next_id]
        generated.append(next_word)

        input_ids = torch.tensor([[next_id]], dtype=torch.long).to(device)

        if next_word in {"</s>", "<end_doc>"}:
            break

    return " ".join(generated)


In [92]:
import torch
import torch.nn.functional as F
import random

def generate_text_with_context(
    model, seed_text, word2idx, idx2word,
    max_sentences=5, max_tokens_per_sentence=40, context_window=5, device='cuda'
):
    model.eval()
    generated = []
    hidden = None

    # Mã hóa seed ban đầu
    input_ids = torch.tensor(
        [word2idx.get(w, word2idx["<unk>"]) for w in seed_text.split()],
        dtype=torch.long
    ).unsqueeze(0).to(device)

    current_sentence = seed_text.split()

    for sent_id in range(max_sentences):
        for _ in range(max_tokens_per_sentence):
            logits, hidden = model(input_ids, hidden)
            probs = F.softmax(logits[0, -1, :], dim=-1)

            # Chọn token có xác suất cao nhất (nếu nhiều token có cùng xác suất → random)
            max_prob = probs.max().item()
            top_indices = (probs == max_prob).nonzero(as_tuple=True)[0]
            if len(top_indices) > 1:
                next_id = random.choice(top_indices.tolist())
            else:
                next_id = top_indices.item()

            next_word = idx2word[next_id]
            current_sentence.append(next_word)

            input_ids = torch.tensor([[next_id]], dtype=torch.long).to(device)

            if next_word in {"</s>", "<end_doc>"}:
                break

        generated.extend(current_sentence)

        # Nếu gặp token kết thúc tài liệu → dừng
        if next_word in {"<end_doc>"}:
            break

        # Lấy một phần ngữ cảnh (context_window từ cuối của câu vừa sinh)
        context = current_sentence[-context_window:]
        input_ids = torch.tensor(
            [word2idx.get(w, word2idx["<unk>"]) for w in context],
            dtype=torch.long
        ).unsqueeze(0).to(device)

        current_sentence = []  # bắt đầu câu mới, nhưng giữ hidden

    return " ".join(generated)


In [101]:
output = generate_text_with_context(
    model, "<start_doc> <s> tp hcm tăng_cường thêm xe phục_vụ thí_sinh", word2idx, idx2word,
    max_sentences=10, max_tokens_per_sentence=50, context_window=50
)
print(output)


<start_doc> <s> tp hcm tăng_cường thêm xe phục_vụ thí_sinh tp hcm hôm_nay , ông <unk> , phó phòng thanh_tra sở văn_hóa thông_tin hà_nội , cho biết , theo kế_hoạch , cuối năm 2005 , thanh_tra sở văn_hóa thông_tin hà_nội đã xử_phạt hàng tỷ đồng , tịch_thu hơn 120.000 cuốn sách , hơn 15 tấn sách bán thành_phẩm các loại chưa kịp <unk> 11.000 tờ hóa_đơn tài_chính ... ngoài_ra còn có chương_trình sử_dụng dịch_vụ ăn_uống , giải_khát cao_cấp . </s> <s> ngoài_ra , khách tham_dự festival còn có_thể <unk> nháp rượu vang đà lạt và của một_số nước chuyên sản_xuất sản_phẩm trước_đây . </s> <s> các tour du_lịch được việt_kiều quan_tâm nhất là nghỉ_ngơi cùng gia_đình và khám_phá vẻ đẹp đất_nước . </s> <s> bà tô <unk> , phó giám_đốc công_ty du_lịch bến thành , cho biết thêm : " tàu của anh vẫn còn nằm trên biển , mặc_dù không_thể leo lên đỉnh_cao hơn ngoại_trừ " một fan " ở đây , là một trong những người đầu_tiên lái boeing 777 hạ_cánh xuống sân_bay san_francisco , mang đến cho nhân_dân mỹ hình_ảnh của

In [ ]:
import torch

# Lưu trọng số của mô hình
torch.save(model.state_dict(), "/kaggle/working/model_final.pth")

print("Model đã được lưu tại /kaggle/working/model_final.pth")


In [36]:
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss.item(),
}, "/kaggle/working/checkpoint.pth")

print("Checkpoint đã được lưu!")


Checkpoint đã được lưu!


In [ ]:
import torch

# 1. Khởi tạo lại kiến trúc model (phải giống khi bạn train)
model = LSTMLM(vocab_size).to(device)

# 2. Load trọng số đã lưu
model.load_state_dict(torch.load("model_final.pth", map_location=torch.device("cpu")))

# 3. Đặt chế độ eval
model.eval()

print("Mô hình đã được load thành công và sẵn sàng dùng!")


In [34]:
import torch
import torch.optim as optim
import zipfile

# # Giải nén file zip vào working directory
# with zipfile.ZipFile("/kaggle/input/model_final.zip", 'r') as zip_ref:
#     zip_ref.extractall("/kaggle/working")
device = "cuda" if torch.cuda.is_available() else "cpu"
criterion = nn.CrossEntropyLoss()
# Tạo lại model và optimizer
model = LSTMLM(vocab_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Load checkpoint
checkpoint = torch.load("/kaggle/input/lstm/pytorch/default/1/checkpoint (8).pth", map_location=device)

# Khôi phục trạng thái mô hình và optimizer
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
start_epoch = checkpoint['epoch']
loss_value = checkpoint['loss']

print(f"Khôi phục từ epoch {start_epoch} với loss {loss_value:.4f}")


Khôi phục từ epoch 4 với loss 1.6111


In [35]:
from tqdm import tqdm


n_epochs = 7  # tổng số epoch muốn train đến
for epoch in range(start_epoch, n_epochs + 1):
    model.train()
    total_loss = 0.0

    # tqdm hiển thị tiến độ theo batch
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader),
                        desc=f"Epoch {epoch+1}/{n_epochs}", ncols=100)

    for batch_idx, (x, y) in progress_bar:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()

        logits, _ = model(x)
        loss = criterion(logits.view(-1, vocab_size), y.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Hiển thị loss tạm thời ngay trên thanh tqdm
        progress_bar.set_postfix({"loss": f"{total_loss/(batch_idx+1):.4f}"})

    print(f"Epoch {epoch+1}/{n_epochs} | Average Loss: {total_loss/len(train_loader):.4f}")

Epoch 5/5: 100%|█████████████████████████████████| 9205/9205 [1:33:33<00:00,  1.64it/s, loss=1.5057]


Epoch 5/5 | Average Loss: 1.5057


Epoch 6/5: 100%|█████████████████████████████████| 9205/9205 [1:33:26<00:00,  1.64it/s, loss=1.4471]

Epoch 6/5 | Average Loss: 1.4471


# LSTM mới

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

class LSTMLM2(nn.Module):
    def __init__(self, vocab_size, emb_dim=512, hidden_size=1024, num_layers=2, dropout=0.3):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTM(
            emb_dim, hidden_size, num_layers,
            batch_first=True, dropout=dropout
        )
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.hidden_size = hidden_size
        self.num_layers = num_layers

    def forward(self, input_ids, hx=None):
        emb = self.embed(input_ids)            # (B, T, E)
        out, hx = self.lstm(emb, hx)           # (B, T, H)
        logits = self.fc(out)                  # (B, T, V)
        return logits, hx

    def init_hidden(self, batch_size, device):
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        return (h0, c0)

# 12/10

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Khởi tạo mô hình
model = LSTMLM2(vocab_size=vocab_size, emb_dim=512, hidden_size=1024, num_layers=2, dropout=0.3)
model = model.to(device)

# Loss và optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

n_epochs = 3
batch_size = train_loader.batch_size  # giả sử loader đã định nghĩa batch_size

for epoch in range(n_epochs):
    model.train()
    total_loss = 0.0

    # Khởi tạo hidden state cho batch đầu tiên
    hidden = model.init_hidden(batch_size, device)

    # Dùng tqdm để hiển thị tiến độ
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader),
                        desc=f"Epoch {epoch+1}/{n_epochs}", ncols=100)

    for batch_idx, (x, y) in progress_bar:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        
        # Đảm bảo hidden có đúng kích thước
        if hidden[0].size(1) != x.size(0):
            hidden = model.init_hidden(x.size(0), device)
            
        # Forward pass
        logits, hidden = model(x, hidden)

        # Compute loss
        loss = criterion(logits.view(-1, vocab_size), y.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Detach hidden state để không lan gradient qua toàn bộ lịch sử
        # hidden = (hidden[0].detach(), hidden[1].detach())
        hidden = tuple(h.detach() for h in hidden)

        # Hiển thị loss tạm thời
        progress_bar.set_postfix({"loss": f"{total_loss/(batch_idx+1):.4f}"})

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{n_epochs} | Average Loss: {avg_loss:.4f}")
    


Epoch 1/3: 100%|█████████████████████████████████| 9249/9249 [1:30:54<00:00,  1.70it/s, loss=3.5771]


Epoch 1/3 | Average Loss: 3.5771


Epoch 2/3: 100%|█████████████████████████████████| 9249/9249 [1:30:36<00:00,  1.70it/s, loss=2.5437]


Epoch 2/3 | Average Loss: 2.5437


Epoch 3/3: 100%|█████████████████████████████████| 9249/9249 [1:30:37<00:00,  1.70it/s, loss=2.2673]

Epoch 3/3 | Average Loss: 2.2673


In [17]:
# Lưu checkpoint sau khi train xong
save_path = "model_lstm_lm2.pt"

torch.save({
    'epoch': epoch + 1,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': avg_loss,
    'vocab_size': vocab_size,
}, save_path)

print(f"Model đã được lưu vào: {save_path}")


Model đã được lưu vào: model_lstm_lm2.pt


In [127]:
# Khởi tạo lại mô hình cùng cấu hình
loaded_model = LSTMLM2(vocab_size=vocab_size, emb_dim=512, hidden_size=1024, num_layers=2, dropout=0.3)
loaded_model = loaded_model.to(device)

# Tạo optimizer mới
loaded_optimizer = optim.Adam(loaded_model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# Load checkpoint
checkpoint = torch.load("/kaggle/working/model_lstm_lm2.pt", map_location=device)

loaded_model.load_state_dict(checkpoint['model_state_dict'])
loaded_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
start_epoch = checkpoint['epoch'] + 1
last_loss = checkpoint['loss']

print(f"Model đã load lại từ epoch {start_epoch} với loss {last_loss:.4f}")


Model đã load lại từ epoch 4 với loss 2.2673


In [60]:
from tqdm import tqdm
import torch

# Giả sử bạn đã có:
# loaded_model, loaded_optimizer, criterion, start_epoch, last_loss, train_loader, vocab_size, device

n_epochs_total = 6

for epoch in range(start_epoch, n_epochs_total + 1):
    loaded_model.train()
    total_loss = 0.0

    # Khởi tạo hidden state cho batch đầu tiên
    hidden = loaded_model.init_hidden(train_loader.batch_size, device)

    # Thanh tiến độ tqdm
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader),
                        desc=f"Epoch {epoch}/{n_epochs_total}", ncols=100)

    for batch_idx, (x, y) in progress_bar:
        x, y = x.to(device), y.to(device)
        loaded_optimizer.zero_grad()

        # Nếu batch cuối nhỏ hơn batch_size → reset hidden
        if hidden[0].size(1) != x.size(0):
            hidden = loaded_model.init_hidden(x.size(0), device)

        # Forward
        logits, hidden = loaded_model(x, hidden)

        # Tính loss
        loss = criterion(logits.view(-1, vocab_size), y.view(-1))
        loss.backward()
        loaded_optimizer.step()

        total_loss += loss.item()

        # Ngắt kết nối gradient quá khứ (tránh exploding graph)
        hidden = tuple(h.detach() for h in hidden)

        # Hiển thị loss tạm thời
        progress_bar.set_postfix({"loss": f"{total_loss / (batch_idx + 1):.4f}"})

    # Tính loss trung bình epoch
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch}/{n_epochs_total} | Average Loss: {avg_loss:.4f}")



Epoch 4/6: 100%|█████████████████████████████████| 9249/9249 [1:30:53<00:00,  1.70it/s, loss=2.1312]


Epoch 4/6 | Average Loss: 2.1312


Epoch 5/6: 100%|█████████████████████████████████| 9249/9249 [1:30:49<00:00,  1.70it/s, loss=2.0447]


Epoch 5/6 | Average Loss: 2.0447


Epoch 6/6: 100%|█████████████████████████████████| 9249/9249 [1:30:44<00:00,  1.70it/s, loss=1.9815]

Epoch 6/6 | Average Loss: 1.9815


In [61]:
# Lưu checkpoint
torch.save({
    'epoch': epoch,
    'model_state_dict': loaded_model.state_dict(),
    'optimizer_state_dict': loaded_optimizer.state_dict(),
    'loss': avg_loss,
}, "/kaggle/working/model_lstm_lm2_cont.pt")

print("Checkpoint đã được lưu!\n")


Checkpoint đã được lưu!



In [128]:
# Khởi tạo lại mô hình cùng cấu hình
loaded_model = LSTMLM2(vocab_size=vocab_size, emb_dim=512, hidden_size=1024, num_layers=2, dropout=0.3)
loaded_model = loaded_model.to(device)

# Tạo optimizer mới
loaded_optimizer = optim.Adam(loaded_model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# Load checkpoint
checkpoint = torch.load("/kaggle/working/model_lstm_lm2_cont.pt", map_location=device)

loaded_model.load_state_dict(checkpoint['model_state_dict'])
loaded_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
start_epoch = checkpoint['epoch'] + 1
last_loss = checkpoint['loss']

print(f"Model đã load lại từ epoch {start_epoch} với loss {last_loss:.4f}")


Model đã load lại từ epoch 7 với loss 1.9815


In [129]:
for param_group in loaded_optimizer.param_groups:
    print("Learning rate:", param_group['lr'])

print("Số lượng state lưu trong optimizer:", len(loaded_optimizer.state))


Learning rate: 0.001
Số lượng state lưu trong optimizer: 11


In [105]:
@torch.no_grad()
def generate_from_text(model, prompt_text, word2idx, idx2word,
                       max_tokens_total=100, top_p=0.9, temperature=0.8,
                       max_sentences=4, device="cpu"):
    """
    Sinh văn bản từ prompt_text, giữ hidden state liên tục (phù hợp LSTM).
    Dừng khi đạt:
      - max_tokens_total token, hoặc
      - <end_doc>, hoặc
      - đủ 3–4 câu (max_sentences)
    """
    model.eval()

    # ---- Encode: text → token IDs ----
    start_tokens = [word2idx.get(w, word2idx.get("<unk>", 0)) for w in prompt_text.split()]
    input_ids = torch.tensor([start_tokens], dtype=torch.long, device=device)

    # Khởi tạo hidden state
    hidden = model.init_hidden(batch_size=1, device=device)

    # Chạy prompt để cập nhật hidden
    logits, hidden = model(input_ids, hidden)

    generated = start_tokens.copy()
    sentence_count = 0

    for _ in range(max_tokens_total):
        logits = logits[:, -1, :] / temperature
        probs = torch.softmax(logits, dim=-1)

        # Top-p sampling
        sorted_probs, sorted_idx = torch.sort(probs, descending=True)
        cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
        cutoff_mask = cumulative_probs > top_p
        if cutoff_mask.any():
            cutoff_idx = torch.where(cutoff_mask)[1][0]
            sorted_probs = sorted_probs[:, :cutoff_idx + 1]
            sorted_idx = sorted_idx[:, :cutoff_idx + 1]
        sorted_probs = sorted_probs / sorted_probs.sum(dim=-1, keepdim=True)

        next_token = sorted_idx[0, torch.multinomial(sorted_probs[0], 1)].item()
        generated.append(next_token)

        # Kiểm tra điều kiện dừng
        word = idx2word[next_token]
        if word in {".", "!", "?"}:
            sentence_count += 1
        if word == "<end_doc>" or sentence_count >= max_sentences:
            break

        # Tiếp tục sinh
        input_ids = torch.tensor([[next_token]], device=device)
        logits, hidden = model(input_ids, hidden)

    # ---- Decode ----
    output_words = [idx2word[t] for t in generated]
    return " ".join(output_words)


In [123]:
import torch

@torch.no_grad()
def generate_from_text(model, prompt_text, word2idx, idx2word,
                       max_tokens_total=100, top_p=0.9, temperature=0.8,
                       max_sentences=4, device="cpu", mode="top-p"):
    """
    Sinh văn bản từ prompt_text với chế độ:
      - mode='top-p'  → sampling (random theo xác suất)
      - mode='greedy' → chọn token xác suất cao nhất
    """

    model.eval()
    model.to(device)

    # ---- Encode: text → token IDs ----
    start_tokens = [word2idx.get(w, word2idx.get("<unk>", 0)) for w in prompt_text.split()]
    input_ids = torch.tensor([start_tokens], dtype=torch.long, device=device)

    # Khởi tạo hidden state
    hidden = model.init_hidden(batch_size=1, device=device)

    # Chạy prompt để cập nhật hidden
    logits, hidden = model(input_ids, hidden)

    generated = start_tokens.copy()
    sentence_count = 0

    for _ in range(max_tokens_total):
        logits = logits[:, -1, :] / temperature
        probs = torch.softmax(logits, dim=-1)

        if mode == "greedy":
            # Chọn token có xác suất cao nhất
            next_token = torch.argmax(probs, dim=-1).item()

        elif mode == "top-p":
            # Top-p sampling
            sorted_probs, sorted_idx = torch.sort(probs, descending=True)
            cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
            cutoff_mask = cumulative_probs > top_p
            if cutoff_mask.any():
                cutoff_idx = torch.where(cutoff_mask)[1][0]
                sorted_probs = sorted_probs[:, :cutoff_idx + 1]
                sorted_idx = sorted_idx[:, :cutoff_idx + 1]
            sorted_probs = sorted_probs / sorted_probs.sum(dim=-1, keepdim=True)

            # Phải đảm bảo sampling cũng trên GPU
            next_token = sorted_idx[0, torch.multinomial(sorted_probs[0].to(device), 1)].item()

        else:
            raise ValueError("mode phải là 'top-p' hoặc 'greedy'")

        generated.append(next_token)

        # Kiểm tra điều kiện dừng
        word = idx2word[next_token]
        if word in {".", "!", "?"}:
            sentence_count += 1
        if word == "<end_doc>" or sentence_count >= max_sentences:
            break

        # Tiếp tục sinh token tiếp theo
        input_ids = torch.tensor([[next_token]], device=device)
        logits, hidden = model(input_ids, hidden)

    # ---- Decode ----
    output_words = [idx2word[t] for t in generated]
    return " ".join(output_words)


In [94]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
prompt = "busan lợi_thế khán_giả nhà và cả cái nóng"
print(generate_from_text(loaded_model, prompt, word2idx, idx2word, mode="greedy"))
print(generate_from_text(loaded_model, prompt, word2idx, idx2word, mode="top-p", top_p=0.9))
# print(generate_from_text(loaded_model, prompt, word2idx, idx2word, mode="top_k", top_k=40))

In [ ]:
# prompt_text = "<start_doc> <s> tp hcm tăng_cường thêm xe phục_vụ"
prompt_text = "busan lợi_thế khán_giả nhà và cả cái nóng"
output = generated_text = generate_from_text(model, prompt_text, word2idx, idx2word, device=device)

print(output)


In [76]:
# prompt_text = "<start_doc> <s> tp hcm tăng_cường thêm xe phục_vụ"
prompt_text = "<start_doc> <s> có_thể phải dùng phiếu để"
output = generated_text = generate_from_text(model, prompt_text, word2idx, idx2word, device=device)

print(output)


<start_doc> <s> có_thể phải dùng phiếu để dân giám_sát . </s> <s> bởi lương_thực và các vật_dụng khác cũng có_thể khan_hiếm . </s> <s> các công_trình này đều được xây_dựng khang_trang , sạch_đẹp . </s> <s> tại đây , nhân_ngày quốc_tế thiếu_nhi 1 1 , tại trại_giam hóa lao_động vn sáng nay , cục quản_lý lao_động ngoài nước đã có buổi làm_việc với ban pháp_chế - thường_trực hđnd tp.hcm ; ông phượng cũng cho rằng , những lùm_xùm trong vụ kiện này không_chỉ vì_thế_hệ hiện_nay mà_còn vì chính sự thú_vị mà chỉ cần ăn 1 giờ là như_thế .


In [77]:
import math
from tqdm import tqdm
import torch

@torch.no_grad()
def evaluate_perplexity(model, data_loader, criterion, device="cpu"):
    model.eval()
    total_loss = 0.0
    total_tokens = 0

    hidden = model.init_hidden(batch_size=data_loader.batch_size, device=device)

    # Thanh tiến độ
    progress_bar = tqdm(enumerate(data_loader), total=len(data_loader),
                        desc="Evaluating", ncols=100)

    for batch_idx, (x, y) in progress_bar:
        x, y = x.to(device), y.to(device)

        # Reset hidden nếu batch_size thay đổi (cuối epoch)
        if hidden[0].size(1) != x.size(0):
            hidden = model.init_hidden(x.size(0), device)

        logits, hidden = model(x, hidden)

        # Tính loss cho batch
        loss = criterion(logits.view(-1, logits.size(-1)), y.view(-1))

        # Nhân với số lượng token trong batch
        total_loss += loss.item() * x.numel()
        total_tokens += x.numel()

        # Cắt gradient để tránh tích lũy
        hidden = tuple(h.detach() for h in hidden)

        # Cập nhật tiến độ
        avg_loss = total_loss / max(total_tokens, 1)
        progress_bar.set_postfix({
            "avg_loss": f"{avg_loss:.4f}",
            "PPL": f"{math.exp(min(avg_loss, 100)):.2f}"
        })

    # Trung bình cuối cùng
    avg_loss = total_loss / total_tokens
    # perplexity = math.exp(min(avg_loss, 100))  # tránh tràn số
    perplexity = math.exp(avg_loss)
    print(f"\nAverage loss: {avg_loss:.4f} | Perplexity: {perplexity:.2f}")
    return perplexity


In [78]:
def encode_text(text, word2idx, unk_token="<unk>"):
    tokens = text.strip().split()
    return [word2idx.get(t, word2idx[unk_token]) for t in tokens]

with open("/kaggle/input/data-small/test (1).txt", "r", encoding="utf-8") as f:
    test_text = f.read()

test_ids = encode_text(test_text, word2idx)
print(f"Số lượng token trong test: {len(test_ids):,}")

from torch.utils.data import DataLoader

test_dataset = LSTMDataset(test_ids, seq_len=30)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=True)


Số lượng token trong test: 292,793


In [130]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()

evaluate_perplexity(loaded_model, test_loader, criterion, device=device)

Evaluating: 100%|██████████████████| 1143/1143 [03:30<00:00,  5.42it/s, avg_loss=5.8143, PPL=335.06]


Average loss: 5.8143 | Perplexity: 335.06


335.0642437640533

In [52]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()

evaluate_perplexity(model, test_loader, criterion, device=device)

Evaluating: 100%|██████████████████| 1143/1143 [03:31<00:00,  5.40it/s, avg_loss=5.4743, PPL=238.49]


Average loss: 5.4743 | Perplexity: 238.49


238.4883210939928

In [132]:
num_sent_train = len(train_loader.dataset)
num_sent_test = len(test_loader.dataset)

num_token_train = 0
for x, y in train_loader:
    num_token_train += (y != 0).sum().item()  # nếu có padding = 0

num_token_test = 0
for x, y in test_loader:
    num_token_test += (y != 0).sum().item()

print(f"Train: {num_sent_train} câu, {num_token_train} token")
print(f"Test: {num_sent_test} câu, {num_token_test} token")


Train: 2367791 câu, 70296910 token
Test: 292763 câu, 8618432 token
